# Ch8. Data Wrangling: Join, Combine, and Reshape

## 8.1 Hierarchical Indexing

In [1]:
import pandas as pd
import numpy as np

data = pd.Series(np.random.randn(9),
                index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.397038
   2   -1.864114
   3   -0.003293
b  1   -0.890991
   3    0.117094
c  1    0.613324
   2    0.441097
d  2   -0.309078
   3    0.617228
dtype: float64

In [2]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [3]:
data['b']

1   -0.890991
3    0.117094
dtype: float64

In [4]:
data['b':'c']

b  1   -0.890991
   3    0.117094
c  1    0.613324
   2    0.441097
dtype: float64

In [5]:
data.loc[['b', 'd']]

b  1   -0.890991
   3    0.117094
d  2   -0.309078
   3    0.617228
dtype: float64

In [6]:
data.loc[:,2]  # 2 is inner level selection

a   -1.864114
c    0.441097
d   -0.309078
dtype: float64

In [7]:
data.unstack()

,1,2,3
a,-0.397038,-1.864114,-0.003293
b,-0.890991,NaN,0.117094
c,0.613324,0.441097,NaN
d,NaN,-0.309078,0.617228


In [8]:
data.unstack().stack()

a  1   -0.397038
   2   -1.864114
   3   -0.003293
b  1   -0.890991
   3    0.117094
c  1    0.613324
   2    0.441097
d  2   -0.309078
   3    0.617228
dtype: float64

In [9]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                    columns=[['Ohio', 'Ohio', 'Colorado'],
                             ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [10]:
frame.index.names = ['key1', 'key2']
frame

Ohio     Colorado
          Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [11]:
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [12]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [13]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'],
                          ['Green', 'Red', 'Green']],
                          names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Reordering and Sorting Levels

In [14]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [16]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [21]:
# sort_index(level=0) or sort_index() improve the data selection performance
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by Level

In [22]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [23]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame’s columns

In [24]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [26]:
frame.reset_index()

state key1 key2  Ohio     Colorado
color           Green Red    Green
0        a    1     0   1        2
1        a    2     3   4        5
2        b    1     6   7        8
3        b    2     9  10       11

In [29]:
frame.reset_index().columns

MultiIndex([(    'key1',      ''),
            (    'key2',      ''),
            (    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

In [35]:
frame.reset_index().set_index(('Ohio', 'Green'))

state,key1,key2,Ohio,Colorado
color,,,Red,Green
"(Ohio, Green)",,,,
0,a,1,1,2
3,a,2,4,5
6,b,1,7,8
9,b,2,10,11


## 8.2 Combining and Merging Datasets

* `pandas.merge`: `join` in SQL
* `pandas.concat`: `union` in SQL (stack)
* `combine_first`: splice data to fill in missing values

### Database-Style DataFrame Joins

In [51]:
df1 = pd.DataFrame({'key': ['a', 'b', 'c'], 'value': range(3)})
df2 = pd.DataFrame({'key': ['c', 'b', 'a', 'a'], 'value': range(4)})

In [52]:
# this merges on two keys: key and value
df1.merge(df2)

,key,value
0,b,1


In [42]:
# setting 'on' explicitly is a good practice
pd.merge(df1, df2, on='key')

,key,value_x,value_y
0,a,0,2
1,a,0,3
2,b,1,1
3,c,2,0


### Merging on Index

In [56]:
df1 = pd.DataFrame(index= ['a', 'b', 'c'], data={'value1': range(3)})
df2 = pd.DataFrame(index= ['c', 'b', 'a', 'a'], data={'value2': range(4)})

In [57]:
pd.merge(df1, df2, left_index=True, right_index=True)

,value1,value2
a,0,2
a,0,3
b,1,1
c,2,0


In [59]:
# join: merge with index. left join 
df1.join(df2)

,value1,value2
a,0,2
a,0,3
b,1,1
c,2,0


### Concatenating Along an Axis

In [61]:
arr = np.arange(12).reshape((4,3))
arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [62]:
np.concatenate([arr, arr])

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [63]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  0,  1,  2],
       [ 3,  4,  5,  3,  4,  5],
       [ 6,  7,  8,  6,  7,  8],
       [ 9, 10, 11,  9, 10, 11]])

In [65]:
s1 = pd.Series([0,1,2], index=['a', 'b', 'c'])
s2 = pd.Series([3,4], index=['d', 'e'])
pd.concat([s1, s2])

a    0
b    1
c    2
d    3
e    4
dtype: int64

In [66]:
pd.concat([s1, s2], axis=1)

,0,1
a,0.0,NaN
b,1.0,NaN
c,2.0,NaN
d,NaN,3.0
e,NaN,4.0


In [68]:
s3 = pd.Series([40,50], index=['e', 'f'])
pd.concat([s2, s3], axis=1, join='inner')

,0,1
e,4,40


In [69]:
pd.concat([s1, s2], axis=1, keys=['s1', 's2'])

,s1,s2
a,0.0,NaN
b,1.0,NaN
c,2.0,NaN
d,NaN,3.0
e,NaN,4.0


In [70]:
pd.concat([s1, s2], keys=['s1', 's2'])

s1  a    0
    b    1
    c    2
s2  d    3
    e    4
dtype: int64

In [71]:
pd.concat([s1, s2], keys=['s1', 's2']).unstack()

,a,b,c,d,e
s1,0.0,1.0,2.0,NaN,NaN
s2,NaN,NaN,NaN,3.0,4.0


In [72]:
pd.concat([s1, s2], keys=['s1', 's2']).unstack().unstack()

a  s1    0.0
   s2    NaN
b  s1    1.0
   s2    NaN
c  s1    2.0
   s2    NaN
d  s1    NaN
   s2    3.0
e  s1    NaN
   s2    4.0
dtype: float64

In [73]:
pd.concat([s1, s2], keys=['s1', 's2']).unstack().unstack().unstack()

,s1,s2
a,0.0,NaN
b,1.0,NaN
c,2.0,NaN
d,NaN,3.0
e,NaN,4.0


In [74]:
pd.concat([s1, s2], keys=['s1', 's2']).unstack().unstack().unstack().unstack()

s1  a    0.0
    b    1.0
    c    2.0
    d    NaN
    e    NaN
s2  a    NaN
    b    NaN
    c    NaN
    d    3.0
    e    4.0
dtype: float64

In [75]:
pd.concat([s1, s2], keys=['s1', 's2']).unstack().unstack().unstack().unstack().unstack()

,a,b,c,d,e
s1,0.0,1.0,2.0,NaN,NaN
s2,NaN,NaN,NaN,3.0,4.0


In [76]:
df1 = pd.DataFrame(np.arange(12).reshape((3, 4)), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.arange(6).reshape((2, 3)), columns=['b', 'd', 'a'])
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0,1,2.0,3
1,4,5,6.0,7
2,8,9,10.0,11
3,2,0,NaN,1
4,5,3,NaN,4


In [77]:
pd.concat([df1, df2])

,a,b,c,d
0,0,1,2.0,3
1,4,5,6.0,7
2,8,9,10.0,11
0,2,0,NaN,1
1,5,3,NaN,4


### Combining Data with Overlap

In [81]:
s1 = pd.Series([np.nan, 1, 2])
s2 = pd.Series([0, 10, np.nan])

# s1 and s2 have to have the same dimension
np.where(pd.isnull(s1), s2, s1)

array([0., 1., 2.])

In [82]:
s2.combine_first(s1)

0     0.0
1    10.0
2     2.0
dtype: float64

In [83]:
s2.combine_first(pd.Series([2], index=[2]))

0     0.0
1    10.0
2     2.0
dtype: float64

## 8.3 Reshaping and Pivoting

* `stack`: This "rotates" or pivots from the columns in the data to the rows
* `unstack`: This pivots from the rows into the columns

In [85]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [88]:
# columns to rows
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [89]:
# rows into columns 
# (the innermost level is unstacked by default)
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [90]:
# choose the level to unstack
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [91]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


### Pivoting “Long” to “Wide” Format

In [93]:
data = pd.read_csv('pydata-book/examples/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [94]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')

In [95]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

In [96]:
# why reindex?
data = data.reindex(columns=columns)
data

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2
...,...,...,...
198,13324.600,-3.16,6.0
199,13141.920,-8.79,6.9
200,12925.410,0.94,8.1
201,12901.504,3.37,9.2


In [100]:
# the same
data2 = data[columns]
data2.compare(data) 

Empty DataFrame
Columns: []
Index: []

In [104]:
data.index = periods.to_timestamp(freq='D', how='end')
data.index

DatetimeIndex(['1959-03-31 23:59:59.999999999',
               '1959-06-30 23:59:59.999999999',
               '1959-09-30 23:59:59.999999999',
               '1959-12-31 23:59:59.999999999',
               '1960-03-31 23:59:59.999999999',
               '1960-06-30 23:59:59.999999999',
               '1960-09-30 23:59:59.999999999',
               '1960-12-31 23:59:59.999999999',
               '1961-03-31 23:59:59.999999999',
               '1961-06-30 23:59:59.999999999',
               ...
               '2007-06-30 23:59:59.999999999',
               '2007-09-30 23:59:59.999999999',
               '2007-12-31 23:59:59.999999999',
               '2008-03-31 23:59:59.999999999',
               '2008-06-30 23:59:59.999999999',
               '2008-09-30 23:59:59.999999999',
               '2008-12-31 23:59:59.999999999',
               '2009-03-31 23:59:59.999999999',
               '2009-06-30 23:59:59.999999999',
               '2009-09-30 23:59:59.999999999'],
              dtype=

In [105]:
data.stack()

date                           item   
1959-03-31 23:59:59.999999999  realgdp     2710.349
                               infl           0.000
                               unemp          5.800
1959-06-30 23:59:59.999999999  realgdp     2778.801
                               infl           2.340
                                            ...    
2009-06-30 23:59:59.999999999  infl           3.370
                               unemp          9.200
2009-09-30 23:59:59.999999999  realgdp    12990.341
                               infl           3.560
                               unemp          9.600
Length: 609, dtype: float64

In [106]:
data.stack().reset_index()

,date,item,0
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370
605,2009-06-30 23:59:59.999999999,unemp,9.200
606,2009-09-30 23:59:59.999999999,realgdp,12990.341
607,2009-09-30 23:59:59.999999999,infl,3.560


In [108]:
# where is this 0 column from? 
#   they lost their column names by stacking.
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata # long format

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370
605,2009-06-30 23:59:59.999999999,unemp,9.200
606,2009-09-30 23:59:59.999999999,realgdp,12990.341
607,2009-09-30 23:59:59.999999999,infl,3.560


In [114]:
pivoted = ldata.pivot(index='date', columns='item', values='value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [115]:
ldata.pivot('date', 'item')

value                 
item                           infl    realgdp unemp
date                                                
1959-03-31 23:59:59.999999999  0.00   2710.349   5.8
1959-06-30 23:59:59.999999999  2.34   2778.801   5.1
1959-09-30 23:59:59.999999999  2.74   2775.488   5.3
1959-12-31 23:59:59.999999999  0.27   2785.204   5.6
1960-03-31 23:59:59.999999999  2.31   2847.699   5.2
...                             ...        ...   ...
2008-09-30 23:59:59.999999999 -3.16  13324.600   6.0
2008-12-31 23:59:59.999999999 -8.79  13141.920   6.9
2009-03-31 23:59:59.999999999  0.94  12925.410   8.1
2009-06-30 23:59:59.999999999  3.37  12901.504   9.2
2009-09-30 23:59:59.999999999  3.56  12990.341   9.6

[203 rows x 3 columns]

In [118]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked

value                 
item                           infl    realgdp unemp
date                                                
1959-03-31 23:59:59.999999999  0.00   2710.349   5.8
1959-06-30 23:59:59.999999999  2.34   2778.801   5.1
1959-09-30 23:59:59.999999999  2.74   2775.488   5.3
1959-12-31 23:59:59.999999999  0.27   2785.204   5.6
1960-03-31 23:59:59.999999999  2.31   2847.699   5.2
...                             ...        ...   ...
2008-09-30 23:59:59.999999999 -3.16  13324.600   6.0
2008-12-31 23:59:59.999999999 -8.79  13141.920   6.9
2009-03-31 23:59:59.999999999  0.94  12925.410   8.1
2009-06-30 23:59:59.999999999  3.37  12901.504   9.2
2009-09-30 23:59:59.999999999  3.56  12990.341   9.6

[203 rows x 3 columns]

### Pivoting “Wide” to “Long” Format

In [119]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                    'A': [1, 2, 3],
                    'B': [4, 5, 6],
                    'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [120]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [121]:
df.stack()

0  key    foo
   A        1
   B        4
   C        7
1  key    bar
   A        2
   B        5
   C        8
2  key    baz
   A        3
   B        6
   C        9
dtype: object

In [124]:
reshaped = melted.pivot(index='key', columns='variable', values='value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [125]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [126]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
